# **개요**
이 노트북 파일은 데이터 파일 전처리를 위해 사용되는 노트북입니다.<br>
결과물로는 다음 항목들이 만들어집니다.
- **TalkSetTrain5.tsv :**원본 문장과 분류, 점수쌍이 저장된 tsv 파일입니다.
- **kiwi_pos.pkl :**문장을 Kiwi 형태소 분석한 pkl 파일입니다.
- **input_data.npy :**벡터화, 패딩처리까지 완료된 문장 데이터가 넘파이 배열로 저장되어 있습니다.
- **label_regr.npy :**점수 라벨 데이터가 넘파이 배열로 저장되어 있습니다.
- **label_mult.npy :**분류 라벨 데이터가 라벨 인코더를 통해 인코딩 된 다음 넘파이 배열로 저장되어 있습니다.
- **data_configs.json :**분류 라벨 데이터가 라벨 인코더를 통해 인코딩 된 다음 넘파이 배열로 저장되어 있습니다.


# **0. 준비단계**

##0.0 Functions

In [7]:
def showLengthStat(sentences_series):
    train_length = sentences_series.astype(str).apply(len)

    print(f'댓글 당 형태소 수 최대값: {np.max(train_length)}')
    print(f'댓글 당 형태소 수 최소값: {np.min(train_length)}')
    print(f'댓글 당 형태소 수 평균값: {np.mean(train_length):.2f}')
    print(f'댓글 당 형태소 수 표준편차: {np.std(train_length):.2f}')
    print(f'댓글 당 형태소 수 중간값: {np.median(train_length)}')
    print(f'댓글 당 형태소 수 1사분위: {np.percentile(train_length, 25)}')
    print(f'댓글 당 형태소 수 3사분위: {np.percentile(train_length, 75)}')
    print(f'댓글 당 형태소 수 95% 커버리지: {np.percentile(train_length, 95)}')
    print(f'댓글 당 형태소 수 97% 커버리지: {np.percentile(train_length, 97)}')
    print(f'댓글 당 형태소 수 99% 커버리지: {np.percentile(train_length, 99)}')
    print(f'댓글 당 형태소 수 99.99% 커버리지: {np.percentile(train_length, 99.99)}')
    print(f'댓글 당 형태소 수 99.999% 커버리지: {np.percentile(train_length, 99.999)}')

# 데이터셋 생성하기 
def LSTMDataset(sentence_series, label_series, tok, max_seq_len):
    # 문장 리스트화 
    sentences = sentence_series.tolist()

    # 토크나이저 실행 
    tok.fit_on_texts(sentences)

    # 벡터화 
    sent_vec = tok.texts_to_sequences(sentences)

    # X 패딩 (dtype = float)
    padded_X = pad_sequences(sent_vec, maxlen = max_seq_len)
    padded_X = np.array(padded_X, dtype=float)

    # Y 넘파이로 변환 (dtype = float) 
    Y = np.array(label_series.tolist(), dtype=float)

    # 단어사전
    word_dic = tok.word_index
    # print(len(word_dic))
    # print(word_dic)

    result = {'X':padded_X, 'Y': Y, 'word_dic': word_dic, 'max_seq_len': max_seq_len}

    return result

# 데이터셋 저장하기 
def saveLSTMData(file_names, res, vocab_size, label_class):
    X = res['X']
    Y = res['Y']
    word_dic = res['word_dic']
    max_seq_len = res['max_seq_len']    

    # 데이터 정보 dictionary 생성
    data_configs = {}
    data_configs['vocab'] = word_dic
    data_configs['vocab_size'] = vocab_size
    data_configs['encoded_labels'] = list(label_class)
    data_configs['MAX_SEQ_LEN'] = max_seq_len
    # print(data_configs)

    # 넘파이 파일 저장 
    np.save(open(file_names[0], 'wb'), X)
    np.save(open(file_names[1], 'wb'), Y)
    json.dump(data_configs, open(file_names[2], 'w'), ensure_ascii=False)

    print('저장이 완료되었습니다. ')
    print('-'*50)
    print(f'file_path : {"/".join(file_names[0].split("/")[:-1])}')
    for i, file_path in enumerate(file_names):
        print(f'{i+1}. {file_path.split("/")[-1]}')
    print('-'*50)

## 0.1. 모듈 설치

In [ ]:
# ! pip install Konlpy
# ! pip install tensorflow

## 0.2. 패키지 IMPORT

In [2]:
import os
import re
import json
import pickle

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# file_path = '/content/drive/MyDrive/3조_XXXX/04. 기타자료/data/'
file_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/'

## 0.3. 구글 드라이브 MOUNT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **1. 데이터 전처리** 

## 1.1. 데이터프레임 불러오기

In [3]:
with open(file_path + 'kiwi_dataframe.pkl', 'rb') as f:
    talk_df = pickle.load(f)

# 하나의 morphs는 제외
print(f"하나의 morphs를 가지는 {talk_df.loc[talk_df['MORPHS'].apply(lambda x: len(x)) == 1, :].shape[0]}개의 문장을 제거했습니다.")
talk_df = talk_df.loc[talk_df['MORPHS'].apply(lambda x: len(x)) > 1, :]

# morphs를 스페이스로 연결
talk_df['sentence'] = talk_df['MORPHS'].apply(lambda x: ' '.join(x))
talk_df.to_csv(file_path + 'LSTM_dataframe.csv', index=False)

talk_df = talk_df[['sentence', '분류', '점수']]
talk_df

하나의 morphs를 가지는 343개의 문장을 제거했습니다.


,sentence,분류,점수
0,부랴부랴 오다 아무 안 오다 . 시간 개념 들 없다,권고,10
1,맞다 . 사람 들 진짜 개념 없다,권고,10
2,인방 보다 남자 거르다 것 맞다,권고,5
3,특히 벗방 보다 애 들 진짜 거세 시키 하다,경고,10
4,댓글 빠순이 들 몰려오다 즈그 주인 님 쉴드 치다 꼴 좀 보다,권고,10
...,...,...,...
222240,그것 어떻다 밥그릇 챙기다 살다 수 있다 고민 하 것 훨씬 이득 이다 않다,권고,8
222241,저 나이 쳐 먹다 아이돌 하다 나대다 거 이다 세상 잘 돌아가다,권고,8
222242,저런 애 들 뽑다 주다 이다 하다 그냥 실패 맛 보다 나오다 거 아니다 ㅋ,권고,8
222243,저 여자 오늘 레깅스 입다,비악플,0


## 1.2 새로운 변수 추가(분류1, 분류2, 점수, 점수2)
* 점수2 : 점수 재 분류(5-6 / 7-8 / 9-15)
* 분류1 : 분류를 인코딩
* 분류2 : 악플/비악플 이진 분류로 인코딩


In [4]:
# 점수2
talk_df['점수2'] = talk_df.loc[:,'점수']
talk_df.loc[(talk_df['점수'] >= 5) & (talk_df['점수'] < 7), '점수2'] = 1
talk_df.loc[(talk_df['점수'] >= 7) & (talk_df['점수'] < 9), '점수2'] = 2
talk_df.loc[(talk_df['점수'] >= 9) ,'점수2'] = 3

# 분류1
talk_df['분류1'] = talk_df.loc[:,'분류']
talk_df.loc[talk_df['분류1']=='비악플', '분류1'] = 0
talk_df.loc[talk_df['분류1']=='권고', '분류1'] = 1
talk_df.loc[talk_df['분류1']=='주의', '분류1'] = 2
talk_df.loc[talk_df['분류1']=='경고', '분류1'] = 3

# 분류2
talk_df['분류2'] = talk_df.loc[:,'분류1']
talk_df.loc[talk_df['분류1'] != 0, '분류2'] = 1

model_df = talk_df[['sentence', '점수', '점수2', '분류1', '분류2']]
model_df.columns = ['sentence', '점수1', '점수2', '분류1', '분류2']
model_df

,sentence,점수1,점수2,분류1,분류2
0,부랴부랴 오다 아무 안 오다 . 시간 개념 들 없다,10,3,1,1
1,맞다 . 사람 들 진짜 개념 없다,10,3,1,1
2,인방 보다 남자 거르다 것 맞다,5,1,1,1
3,특히 벗방 보다 애 들 진짜 거세 시키 하다,10,3,3,1
4,댓글 빠순이 들 몰려오다 즈그 주인 님 쉴드 치다 꼴 좀 보다,10,3,1,1
...,...,...,...,...,...
222240,그것 어떻다 밥그릇 챙기다 살다 수 있다 고민 하 것 훨씬 이득 이다 않다,8,2,1,1
222241,저 나이 쳐 먹다 아이돌 하다 나대다 거 이다 세상 잘 돌아가다,8,2,1,1
222242,저런 애 들 뽑다 주다 이다 하다 그냥 실패 맛 보다 나오다 거 아니다 ㅋ,8,2,1,1
222243,저 여자 오늘 레깅스 입다,0,0,0,0


## 1.3. 악플만 있는 데이터

In [5]:
troll_df = model_df.loc[talk_df['분류2'] != 0,:]
troll_df = troll_df.reset_index().drop(columns='index')
troll_df

,sentence,점수1,점수2,분류1,분류2
0,부랴부랴 오다 아무 안 오다 . 시간 개념 들 없다,10,3,1,1
1,맞다 . 사람 들 진짜 개념 없다,10,3,1,1
2,인방 보다 남자 거르다 것 맞다,5,1,1,1
3,특히 벗방 보다 애 들 진짜 거세 시키 하다,10,3,3,1
4,댓글 빠순이 들 몰려오다 즈그 주인 님 쉴드 치다 꼴 좀 보다,10,3,1,1
...,...,...,...,...,...
112659,오다 10 월 모기 물리다 모기 들 모으다 놓다 불 싸지르다 싶다 .,9,3,3,1
112660,그것 어떻다 밥그릇 챙기다 살다 수 있다 고민 하 것 훨씬 이득 이다 않다,8,2,1,1
112661,저 나이 쳐 먹다 아이돌 하다 나대다 거 이다 세상 잘 돌아가다,8,2,1,1
112662,저런 애 들 뽑다 주다 이다 하다 그냥 실패 맛 보다 나오다 거 아니다 ㅋ,8,2,1,1


## 1.4. 텍스트 길이 살펴보기

In [8]:
# 전체 데이터 -> MAX_SEQ_LEN = 57
showLengthStat(talk_df['sentence'])

댓글 당 형태소 수 최대값: 312
댓글 당 형태소 수 최소값: 3
댓글 당 형태소 수 평균값: 22.95
댓글 당 형태소 수 표준편차: 10.75
댓글 당 형태소 수 중간값: 21.0
댓글 당 형태소 수 1사분위: 15.0
댓글 당 형태소 수 3사분위: 28.0
댓글 당 형태소 수 95% 커버리지: 42.0
댓글 당 형태소 수 97% 커버리지: 47.0
댓글 당 형태소 수 99% 커버리지: 57.0
댓글 당 형태소 수 99.99% 커버리지: 137.62019999994664
댓글 당 형태소 수 99.999% 커버리지: 222.0872699993779


In [9]:
# 악플 데이터 -> MAX_SEQ_LEN = 62
showLengthStat(troll_df['sentence'])

댓글 당 형태소 수 최대값: 312
댓글 당 형태소 수 최소값: 3
댓글 당 형태소 수 평균값: 26.17
댓글 당 형태소 수 표준편차: 11.24
댓글 당 형태소 수 중간값: 24.0
댓글 당 형태소 수 1사분위: 19.0
댓글 당 형태소 수 3사분위: 32.0
댓글 당 형태소 수 95% 커버리지: 46.84999999999127
댓글 당 형태소 수 97% 커버리지: 51.0
댓글 당 형태소 수 99% 커버리지: 62.0
댓글 당 형태소 수 99.99% 커버리지: 158.8043999997899
댓글 당 형태소 수 99.999% 커버리지: 243.72065999975894


# =======================================

# 2. **MODEL 데이터 저장**

## 2.1. Model 1: 전체 데이터 다중분류(비악플/권고/주의/경고)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 57

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(model_df['sentence'], model_df['분류1'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[   0.,    0.,    0., ...,  459.,    5.,   14.],
       [   0.,    0.,    0., ...,   17.,  459.,   14.],
       [   0.,    0.,    0., ...,  621.,    9.,   53.],
       ...,
       [   0.,    0.,    0., ...,    6.,   20.,  209.],
       [   0.,    0.,    0., ...,   73., 1036.,  116.],
       [   0.,    0.,    0., ...,   36.,    6.,   19.]])

In [ ]:
res['Y']

array([1., 1., 1., ..., 1., 0., 3.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data1.npy', data_path + 'label_data1.npy', data_path + 'data_configs1.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['비악플','권고','주의','경고'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data1.npy
2. label_data1.npy
3. data_configs1.json
--------------------------------------------------


## 2.2. Model 2: 전체 데이터 이중분류(비악플/악플)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 57

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(model_df['sentence'], model_df['분류2'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[   0.,    0.,    0., ...,  459.,    5.,   14.],
       [   0.,    0.,    0., ...,   17.,  459.,   14.],
       [   0.,    0.,    0., ...,  621.,    9.,   53.],
       ...,
       [   0.,    0.,    0., ...,    6.,   20.,  209.],
       [   0.,    0.,    0., ...,   73., 1036.,  116.],
       [   0.,    0.,    0., ...,   36.,    6.,   19.]])

In [ ]:
res['Y']

array([1., 1., 1., ..., 1., 0., 1.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data2.npy', data_path + 'label_data2.npy', data_path + 'data_configs2.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['비악플','악플'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data2.npy
2. label_data2.npy
3. data_configs2.json
--------------------------------------------------


## 2.3. Model 3: 전체 데이터 점수 회귀(5 ~ 15)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 57

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(model_df['sentence'], model_df['점수1'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[   0.,    0.,    0., ...,  459.,    5.,   14.],
       [   0.,    0.,    0., ...,   17.,  459.,   14.],
       [   0.,    0.,    0., ...,  621.,    9.,   53.],
       ...,
       [   0.,    0.,    0., ...,    6.,   20.,  209.],
       [   0.,    0.,    0., ...,   73., 1036.,  116.],
       [   0.,    0.,    0., ...,   36.,    6.,   19.]])

In [ ]:
res['Y']

array([10., 10.,  5., ...,  8.,  0.,  9.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data3.npy', data_path + 'label_data3.npy', data_path + 'data_configs3.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=[])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data3.npy
2. label_data3.npy
3. data_configs3.json
--------------------------------------------------


## 2.4. Model 4: 전체 데이터 점수 다중분류(비악플/상/중/하)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 57

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(model_df['sentence'], model_df['점수2'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[   0.,    0.,    0., ...,  459.,    5.,   14.],
       [   0.,    0.,    0., ...,   17.,  459.,   14.],
       [   0.,    0.,    0., ...,  621.,    9.,   53.],
       ...,
       [   0.,    0.,    0., ...,    6.,   20.,  209.],
       [   0.,    0.,    0., ...,   73., 1036.,  116.],
       [   0.,    0.,    0., ...,   36.,    6.,   19.]])

In [ ]:
res['Y']

array([3., 3., 1., ..., 2., 0., 3.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data4.npy', data_path + 'label_data4.npy', data_path + 'data_configs4.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['비악플','하','중','상'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data4.npy
2. label_data4.npy
3. data_configs4.json
--------------------------------------------------


## ================================

## 2.5. Model 5: 악플 데이터 다중분류(권고/주의/경고)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 62

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(troll_df['sentence'], troll_df['분류1'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[  0.,   0.,   0., ..., 311.,   3.,  10.],
       [  0.,   0.,   0., ...,  17., 311.,  10.],
       [  0.,   0.,   0., ..., 449.,   8.,  52.],
       ...,
       [  0.,   0.,   0., ..., 217.,  58., 570.],
       [  0.,   0.,   0., ...,   6.,  20., 136.],
       [  0.,   0.,   0., ...,  32.,   6.,  27.]])

In [ ]:
res['Y'] = res['Y'] - 1
res['Y']

array([0., 0., 0., ..., 0., 0., 2.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data5.npy', data_path + 'label_data5.npy', data_path + 'data_configs5.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['권고','주의','경고'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data5.npy
2. label_data5.npy
3. data_configs5.json
--------------------------------------------------


## 2.6. Model 6: 악플 데이터 점수 다중분류(상/중/하)

In [ ]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 62

In [ ]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(troll_df['sentence'], troll_df['점수2'], tok, max_seq_len=MAX_SEQ_LEN)

In [ ]:
res['X']

array([[  0.,   0.,   0., ..., 311.,   3.,  10.],
       [  0.,   0.,   0., ...,  17., 311.,  10.],
       [  0.,   0.,   0., ..., 449.,   8.,  52.],
       ...,
       [  0.,   0.,   0., ..., 217.,  58., 570.],
       [  0.,   0.,   0., ...,   6.,  20., 136.],
       [  0.,   0.,   0., ...,  32.,   6.,  27.]])

In [ ]:
res['Y'] = res['Y'] - 1
res['Y']

array([2., 2., 0., ..., 1., 1., 2.])

In [ ]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data6.npy', data_path + 'label_data6.npy', data_path + 'data_configs6.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['하','중','상'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data6.npy
2. label_data6.npy
3. data_configs6.json
--------------------------------------------------


## 2.7. Model 7: 악플 데이터 점수 회귀(5~15)

In [10]:
# 99% 커버리지 사용
MAX_SEQ_LEN = 62

In [11]:
# 데이터 생성하기
tok = Tokenizer(oov_token="<OOV>")
res = LSTMDataset(troll_df['sentence'], troll_df['점수1'], tok, max_seq_len=MAX_SEQ_LEN)

In [12]:
res['X']

array([[  0.,   0.,   0., ..., 311.,   3.,  10.],
       [  0.,   0.,   0., ...,  17., 311.,  10.],
       [  0.,   0.,   0., ..., 449.,   8.,  52.],
       ...,
       [  0.,   0.,   0., ..., 217.,  58., 570.],
       [  0.,   0.,   0., ...,   6.,  20., 136.],
       [  0.,   0.,   0., ...,  32.,   6.,  27.]])

In [13]:
res['Y']

array([10., 10.,  5., ...,  8.,  8.,  9.])

In [14]:
# 데이터 저장하기
data_path = '/content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test/'
file_names = [data_path + 'input_data7.npy', data_path + 'label_data7.npy', data_path + 'data_configs7.json']
saveLSTMData(file_names=file_names, res=res, vocab_size=100000, label_class=['권고','주의','경고'])

저장이 완료되었습니다. 
--------------------------------------------------
file_path : /content/drive/MyDrive/Colab Notebooks/3조_XXXX/04. 기타자료/data/lstm_test
1. input_data7.npy
2. label_data7.npy
3. data_configs7.json
--------------------------------------------------
